In [34]:
import pandas as pd   
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
import os

In [6]:
start_data_11_iter_1 = data_11_iter_1['timestamp'].max()
stop_data_11_iter_1 = data_11_iter_2['timestamp'].min()

In [11]:
start_datetime = datetime.strptime(start_data_11_iter_1, '%Y-%m-%d %H:%M:%S.%f')
stop_datetime = datetime.strptime(stop_data_11_iter_1, '%Y-%m-%d %H:%M:%S.%f')
duration = stop_datetime - start_datetime
print("Experiment duration {} ".format(duration))
print("Number of rows {}".format(agg_data_11_iter_1.shape[0]))
print("Number of rows {}".format(agg_data_11_iter_2.shape[0]))

Experiment duration 0:00:00.414602 
Number of rows 1007
Number of rows 1007


In [83]:
def get_subdirectories(directory=""):
    subdirectories = []
    p = Path("./../experiments_data/" + directory)
    for item in p.glob('*/'):
        if item.suffix not in (['.csv', '.zip']):
            subdirectories.append(directory + "/" + item.name)
    return subdirectories

In [84]:
nodes_directories = get_subdirectories()
nodes_directories

['/11_experiments_data_2021_05_05_21_37',
 '/12_experiments_data_2021_05_05_21_37',
 '/13_experiments_data_2021_05_05_21_37',
 '/20_experiments_data_2021_05_05_21_37']

In [91]:
data_directories = []
for directory in nodes_directories:
    cur_node_subdirectories = get_subdirectories(directory)
    data_directories.append(cur_node_subdirectories)

data_directories = [item for sublist in data_directories for item in sublist]
data_directories

['/11_experiments_data_2021_05_05_21_37/countDistinctTicketNumber',
 '/12_experiments_data_2021_05_05_21_37/countDistinctTicketNumber',
 '/13_experiments_data_2021_05_05_21_37/averageTemperatureByDeviceIdSeason',
 '/13_experiments_data_2021_05_05_21_37/countDistinctTicketNumber',
 '/20_experiments_data_2021_05_05_21_37/countDistinctTicketNumber']

In [121]:
for experiment_directory in data_directories[0:1]:
    path = "./../experiments_data" + experiment_directory
    p = Path(path)
    iteration = 1
    agg_base_data = pd.DataFrame()

    for file in p.glob('*.csv'):
        if iteration == 1:
            base_data = pd.read_csv(path + "/" + file.name)
            agg_base_data = base_data.groupby('timestamp', as_index=False).agg({"CPU": "sum", "RAM": "sum"})

            fig = plt.figure(figsize=(20,10))
            plt.plot(agg_base_data.index, agg_base_data.CPU, 'r.-')
            plt.savefig(path + "/" + str(iteration) + "CPU.png")
            plt.close(fig)
        else:
            new_data = pd.read_csv(path + "/" + file.name)
            agg_new_data = new_data.groupby('timestamp', as_index=False).agg({"CPU": "sum", "RAM": "sum"})

            fig = plt.figure(figsize=(20,10))
            plt.plot(agg_new_data.index, agg_new_data.CPU, 'r.-')
            plt.savefig(path + "/" + str(iteration) + "CPU.png")
            plt.close(fig)
            
            concat_data = pd.concat((agg_base_data, agg_new_data))
            by_rows_index = concat_data.groupby(concat_data.index)
            agg_base_data = by_rows_index.mean()

        iteration += 1

    agg_base_data.to_csv(path + "/data.csv", index=False)

    fig = plt.figure(figsize=(20,10))
    plt.plot(agg_base_data.index, agg_base_data.CPU, 'r.-')
    plt.savefig(path + "/" + "avg_CPU.png")
    plt.close(fig)


 1153 rows
